In [352]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 
import seaborn as sns

from sklearn import (linear_model, metrics, neural_network, pipeline, preprocessing, model_selection)

In [353]:
"""
Load data.
Change path variable to your location of CSV files.
"""

path = "./data/"

df_zones = pd.read_csv(path + "zona_censal_comuna.csv")
df_demos = pd.read_csv(path + "demo_comunas.csv")
df_inf = pd.read_csv(path + "dp1_contagiados_por_comuna.csv")
df_mob1 = pd.read_csv(path + "od_zones_weeks_20200301.enc.csv")
df_mob2 = pd.read_csv(path + "od_zones_weeks_20200401.enc.csv")
df_mob = pd.concat([df_mob1, df_mob2])

### Census zones & Communes

In [410]:
"""
Census zone information (only for Santiago).

Relevant columns:
    GEOCODIGO = census zone id
    COMUNA    = commune id
"""
df_zones

,OBJECTID,REGION,PROVINCIA,NOM_COMUNA,COMUNA,GEOCODIGO
0,1,13,134,PAINE,13404,13404011003
1,2,13,134,PAINE,13404,13404011002
2,3,13,134,PAINE,13404,13404061004
3,4,13,134,PAINE,13404,13404061001
4,5,13,134,PAINE,13404,13404061002
...,...,...,...,...,...,...
1860,1861,13,136,EL MONTE,13602,13602031001
1861,1862,13,136,TALAGANTE,13601,13601021001
1862,1863,13,136,TALAGANTE,13601,13601021002
1863,1864,13,135,ALHUÉ,13502,13502011001


In [357]:
"""
Create dictionaries for (standardized) ID lookups of zones and communes.
"""
# Enumerate communes with standardized IDs (from 0 to 50)
dict_commune_sid = {commune:sid for sid, commune in enumerate(df_zones.COMUNA.unique())}

# Dictionary mapping census zone id to its commune standardized id
dict_zone_commune_sid = {zone:dict_commune_sid[commune] for zone, commune in zip(df_zones.GEOCODIGO, df_zones.COMUNA)}    

### Mobility data

In [358]:
"""
Logs of averaged amount of movement of individuals across census zones.

Relevant columns:
    mean_population_zone = weekly mean of daily number of individuals
                                that spend majority of their day in census zone [code_zone]
                                that live in census zone [home_zone]
    code_zone            = visiting census zone
    home_zone            = home census zone
    week                 = week of year
    time_block           = T1 - 10:00-13:00, T2 - 14:00-17:00
"""
df_mob

,mean_population_zone,code_zone,week,time_block,home_zone
0,154.800000,13124061022,9,T1,13124061005
1,14.200000,13101081001,9,T1,13126031003
2,12.800000,13123021005,9,T1,13120041002
3,26.800000,13114021002,9,T1,13114061001
4,44.200000,13114131005,9,T1,13114031002
...,...,...,...,...,...
4642958,1.666667,13130031001,17,T2,13112051010
4642959,1.666667,13119011005,17,T2,13106051001
4642960,1.666667,13301011001,17,T1,13104051001
4642961,1.666667,13119201001,17,T2,13124061005


In [377]:
"""
Create dataframe of edge information: (home zone, visiting zone, week, mean population)
Aggregate information into communes.
"""
t = df_mob.week.min()    # earliest week available
get_zone_com_sid = lambda i: dict_zone_commune_sid[i]

df_edge_list = df_mob[["home_zone","code_zone","week","mean_population_zone"]]
df_edge_list.columns = ['home_com', 'travel_com', 'week', 'mean_pop']
df_edge_list.loc[:,"home_com"] = df_edge_list.loc[:,"home_com"].apply(get_zone_com_sid)
df_edge_list.loc[:,"travel_com"] = df_edge_list.loc[:,"travel_com"].apply(get_zone_com_sid)
df_edge_list.loc[:,"week"] = df_edge_list.loc[:,"week"] - t    # zero index weeks

# Aggregate data by summing mean population of same edges in same week. Sum data from both time blocks.
df_edge_list = df_edge_list.groupby(['home_com','travel_com','week']).agg({'mean_pop':'sum'}).reset_index()

In [378]:
"""
Create mobility matrix: A[i,j,t] 
"""
T = df_mob.week.max() - df_mob.week.min() + 1
N = len(dict_commune_sid)
A = np.zeros((N, N, T))

edge_list = zip(df_edge_list.home_com, df_edge_list.travel_com, df_edge_list.week, df_edge_list.mean_pop)
for edge in edge_list:
    i, j, t, a_ijt = edge
    A[i,j,t] += a_ijt

In [379]:
# Mobility matrix over 51 communes in Santiago over 9 time periods (weeks 9-17).
print(A.shape)

# Matrix is not sparse
print(np.count_nonzero(A), N*N*T)

(51, 51, 9)
21949 23409


### Infections data

In [380]:
"""
Logs of cumulative number of infections in each commune in Chile over differently spaced out days in weeks 14-19.

Relevant columns:
    Codigo comuna          = commune ID
    Date (e.g. 2020-03-30) = number of confirmed cases of COVID19 on that day.
"""
df_inf

,Unnamed: 0,Region,Codigo region,Comuna,Codigo comuna,Poblacion,2020-03-30,2020-04-01,2020-04-03,2020-04-06,...,2020-04-13,2020-04-15,2020-04-17,2020-04-20,2020-04-24,2020-04-27,2020-05-01,2020-05-04,2020-05-08,Tasa
0,0,Arica y Parinacota,15,Arica,15101,247552.0,6.0,6.0,12.0,41.0,...,115.0,124.0,134.0,166.0,224.0,270.0,297.0,310.0,328.0,132.5
1,1,Arica y Parinacota,15,Camarones,15102,1233.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,Arica y Parinacota,15,General Lagos,15202,810.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,Arica y Parinacota,15,Putre,15201,2515.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,Tarapacá,1,Alto Hospicio,1107,129999.0,0.0,0.0,0.0,5.0,...,14.0,15.0,16.0,27.0,39.0,55.0,77.0,128.0,161.0,123.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
341,341,Magallanes,12,Punta Arenas,12101,141984.0,29.0,87.0,143.0,203.0,...,387.0,416.0,470.0,516.0,581.0,623.0,685.0,744.0,825.0,581.1
342,342,Magallanes,12,Rio Verde,12103,211.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
343,343,Magallanes,12,San Gregorio,12104,681.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,146.8
344,344,Magallanes,12,Timaukel,12303,282.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [385]:
"""
Aggregate infections data into weeks.
Only keep data for communes in Santiago.
"""
get_com_sid = lambda i: dict_commune_sid[i] if i in dict_commune_sid else pd.NaT
df_y = pd.DataFrame()
df_y["com"] = df_inf["Codigo comuna"]
df_y.loc[:,"com"] = df_y.loc[:,"com"].apply(get_com_sid)
df_y["week14"] = df_inf["2020-03-30"] + df_inf["2020-04-01"] + df_inf["2020-04-03"]
df_y["week15"] = df_inf["2020-04-06"] + df_inf["2020-04-08"] + df_inf["2020-04-10"]
df_y["week16"] = df_inf["2020-04-13"] + df_inf["2020-04-15"] + df_inf["2020-04-17"]
df_y["week17"] = df_inf["2020-04-20"] + df_inf["2020-04-24"]
df_y["week18"] = df_inf["2020-04-27"] + df_inf["2020-05-01"]
df_y["week19"] = df_inf["2020-05-04"] + df_inf["2020-05-08"]
df_y = df_y.dropna().sort_values("com")

# Get time-evolving cumulative infections vector
y = df_y.drop("com",1).to_numpy()

In [386]:
# Cumulative infections over 51 communes in Santiago over 6 time periods (weeks 14-19)
y.shape

(51, 6)

### Demographics

In [409]:
"""
Demographics information for all communes in Chile.
"""
df_demos.head()

,COMUNA,NOM_COMUNA,Densidad_,SUPERFICIE__KM2_,T_POB,T_HOM,T_MUJ,T_VIV,por_muj,por_hom,pers_viv,viv_km2_total
0,13101,SANTIAGO,17483.935547,23.135237,404495,206678,197817,193628,0.489047,0.510953,2.089032,8369.397823
1,13120,ÑUÑOA,12353.292969,16.856802,208237,95409,112828,92248,0.541825,0.458175,2.257361,5472.449641
2,13108,INDEPENDENCIA,13633.545898,7.355460,100281,49186,51095,36666,0.509518,0.490482,2.734986,4984.868268
3,13123,PROVIDENCIA,9870.609375,14.394146,142079,65710,76369,70965,0.537511,0.462489,2.002100,4930.129245
4,13117,LO PRADO,14671.995117,6.560049,96249,46799,49450,29526,0.513772,0.486228,3.259805,4500.881303


In [407]:
"""
Get demographic information for communes in Santiago.
"""
df_C = df_demos.copy()
df_C.loc[:,"COMUNA"] = df_C.loc[:,"COMUNA"].apply(get_com_sid)
df_C = df_C.dropna().sort_values("COMUNA")
df_C.drop(["COMUNA", "NOM_COMUNA"], axis=1, inplace=True)

# Get demographics feature matrix
C = df_C.to_numpy()

In [408]:
# Demographics features over 51 communes in Santiago
print(C.shape)

(51, 10)


### Feature Extraction

In [317]:
# Feature extraction
def extract_features(A, y, C):
    """
    Extracts feature vectors for each sequential input.
    Returns a matrix of all features X = [X_1 X_2 ... X_T] where X_t is matrix of features for data at time t.
    """
    
    N = 51
    L = 15
    T = 4
    X = np.zeros((N, T*L))
    for t in range(T):
        At = A[:,:,t]
        yt = y[:,t]
        
        x1 = np.ones(N)
        x2 = np.diag(At)
        x3 = np.dot(At.T, np.ones(N))
        x4 = yt
        x5 = np.dot(At.T, yt)
        
        Xt = np.stack((x1,x2,x3,x4,x5), axis=-1)
        Xt = np.column_stack((Xt, C))
        
        X[:,t*L:(t+1)*L] = Xt
    
    return X

def collect_data(X, y):
    L = 15
    T = 5
    K = 1
    Z = X[:,:K*L]
    for t in range(K,T-1):
        Zt = X[:,t*L:(t+K)*L]
        Z = np.row_stack((Z,Zt))
    
    Y = y[:,0]
    for t in range(1, T-1):
        Y = np.concatenate((Y,y[:,t]),axis=None)
        
    return Z, Y
        

In [341]:
### Train-validation-test dataset split

def timeseries_train_test_split(Z, Y):
    """
    Splits the sample into a train and test data.
    """
    T = 4
    K = 1
    N = 51
    L = 15
    X_train = X[:(T-K-1)*N,:]
    X_test = X[(T-K-1)*N:(T-K)*N,:]
    y_train = Y[K*N:(T-K)*N]
    y_test = Y[(T-K)*N:]
    
    return X_train, y_train, X_test, y_test

In [342]:
### Plot forecast
def train_test_plot(model, X_train, X_test):
    """
    This will plot the actual values of data against the one fitted by the model.
    """
    return

### Linear Regression

### LASSO

In [343]:
# LASSO

Z = extract_features(A, y, C)
X, Y = collect_data(Z, y)
X_train, Y_train, X_test, Y_test = timeseries_train_test_split(X, Y)

print(X.shape, Y.shape)
print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

(204, 15) (204,)
(102, 15) (51, 15) (102,) (51,)


In [348]:
lasso = linear_model.LassoCV(cv=model_selection.TimeSeriesSplit(), 
                             alphas=None, tol = 10000, normalize=True) 


cv_lasso = lasso.fit(X_train, Y_train)
optimal_alpha = cv_lasso.alpha_

lasso2 = linear_model.Lasso(alpha=optimal_alpha, normalize=True)
lasso2.fit(X_train, Y_train)


reg = linear_model.LinearRegression()
reg.fit(X_train, Y_train)


# train_test_plot(lasso2, X_train, X_test) 

/Users/gael/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:784: FutureWarning: The default value of n_split will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(NSPLIT_WARNING, FutureWarning)
/Users/gael/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 949.5442838562594, tolerance: 554.0275490196079
  positive)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [349]:
# MSE
metrics.mean_squared_error(Y_test, lasso2.predict(X_test))

41272.8609053205

In [350]:
metrics.mean_squared_error(Y_test, reg.predict(X_test))

48178.24594707671

In [ ]:
# Feature selection
lasso_coefs = pd.DataFrame({"features":list(X_train), "coef": lasso2.coef_})
lasso_coefs = lasso_coefs[lasso_coefs.coef != 0.0]
lasso_coefs.sort_values("coef", ascending=False)

###### The following implementations are not finished yet...

### XGBoost

In [ ]:
X_train, y_train, X_test, y_test = timeseries_train_test_split(X=X, y=y)


scaler = preprocessing.StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

xgb = XGBRegressor()
xgb.fit(X_train_scaled, y_train)

X_test_scaled = scaler.transform(X_test)

train_test_plot(model=xgb, X_train=X_train_scaled, X_test=X_test_scaled)


### Neural Network

In [ ]:
X_train, y_train, X_test, y_test = timeseries_train_test_split(X=X, y=y)

reg = neural_network.MLPRegressor(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)
reg.fit(X, y)

train_test_plot(model=reg, X_train=X_train_scaled, X_test=X_test_scaled)